In [1]:
import pandas as pd
import pickle
# %pip install pyarrow

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[((df.duration >= 1) & (df.duration <= 60))]

    categorical = ['PULocationID', 'DOLocationID']
    # categorical = []
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe("./data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("./data/green_tripdata_2025-02.parquet")

In [4]:
len(df_train), len(df_val)

(46307, 44218)

In [5]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance', 'fare_amount']

dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dict)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

print(mean_squared_error(y_val, y_pred), mean_absolute_error(y_val, y_pred))

29.78915149041427 2.884789395636988


In [8]:
with open("models/lin_reg.bin", "wb") as lr_f_out:
    pickle.dump((dv, lr), lr_f_out)

In [9]:
ls = Lasso(alpha=0.0001)
ls.fit(X_train, y_train)
y_pred = ls.predict(X_val)

print(mean_squared_error(y_val, y_pred), mean_absolute_error(y_val, y_pred))

29.80222092462341 2.882002468352985


In [10]:
with open("models/lin_reg.bin", "wb") as ls_f_out:
    pickle.dump((dv, ls), ls_f_out)

In [11]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_val)

print(mean_squared_error(y_val, y_pred), mean_absolute_error(y_val, y_pred))

29.775923570160266 2.8805038045253863


In [12]:
with open("models/lin_reg.bin", "wb") as ri_f_out:
    pickle.dump((dv, ridge), ri_f_out)